In [1]:
import numpy as np  # linear algebra
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import cv2
import random
import os
import pandas as pd

import sklearn
from PIL import Image
import random
import torch
from detectron2.structures import BoxMode
# import tensorflow as tf
# from tensorflow.keras.layers import Input
# from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
# from tensorflow.keras.preprocessing import image
# from tensorflow.keras.models import Model
# from tensorflow.keras import backend as K 
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.optimizers import Adam, SGD, Adagrad, Adadelta, RMSprop
# from tensorflow.keras.layers import Dropout, Flatten, Activation
# from tensorflow.keras.layers import Conv2D, MaxPooling2D
import warnings

ModuleNotFoundError: No module named 'detectron2'

In [ ]:
#Set data path for our dataset that we will be using
dataPath = 'C:/Users/Austin/HCL_TrafficLight/LISATrafficSet'
sorted(os.listdir(dataPath))

In [ ]:
#Set path for our annotated training folders
dayTrainPath = dataPath + '/Annotations/Annotations/dayTrain'
nightTrainPath = dataPath + '/Annotations/Annotations/nightTrain'

In [ ]:
# Combine all the annotated files in all the Day Training folders 
# and set 'dayTime' feature to 1 since all during day time

trainDayTime = []
for dayFileName in sorted(os.listdir(dayTrainPath)):
    if 'dayClip' in dayFileName:
        df = pd.read_csv(os.path.join(dayTrainPath, dayFileName,'frameAnnotationsBOX.csv'),sep=';')
        trainDayTime.append(df)
    else:
        continue
trainDayTime_df = pd.concat(trainDayTime, axis=0)
trainDayTime_df['dayTime'] = 1

In [ ]:
#Check to make sure that 'dayTime' is set to 1
trainDayTime_df

In [ ]:
# Combine all the annotated files in all the Night Training folders 
# and set 'dayTime' feature to 0 since all during night time

trainNightTime = []
for nightFileName in sorted(os.listdir(nightTrainPath)):
    if 'nightClip' in nightFileName:
        df = pd.read_csv(os.path.join(nightTrainPath, nightFileName,'frameAnnotationsBOX.csv'),sep=';')
        trainNightTime.append(df)
    else:
        continue
trainNightTime_df = pd.concat(trainNightTime, axis=0)
trainNightTime_df['dayTime'] = 0

In [ ]:
#Check to make sure that 'dayTime' is set to 0 since night time dataset
trainNightTime_df

In [ ]:
# Now that we have all of the day and night annotations combined respectively, we will combine 
# the two into create one large dataset
combined_df = pd.concat([trainDayTime_df, trainNightTime_df], axis=0)

In [ ]:
# Checking to see if we have any missing values in our dataset
combined_df.isnull().sum()

In [ ]:
# Four of the columns in our dataframe appear to have the same values, we are checking here to confirm that they are the same
print(df['Origin file'].equals(df['Origin track']))
print(df['Origin frame number'].equals(df['Origin track frame number']))

In [ ]:
# Since they are the same values, we can drop the duplicate columns that we don't need
combined_df = combined_df.drop(['Origin file','Origin track','Origin frame number','Origin track frame number'], axis=1)

In [ ]:
combined_df.columns = ['id', 'signal', 'left_x', 'left_y','right_x','right_y','dayTime']
combined_df.head()

In [ ]:
combined_df['signal'].unique()

In [ ]:
# Change classes to an integer
label_to_idx = {'go':1, 'warning':2, 'stop': 3}
idx_to_label = {v:k for k,v in label_to_idx.items()}


def changeSignal(rowLabel):
    if rowLabel == 'go' or rowLabel == 'goLeft':
        return 1
    elif rowLabel == 'warning' or rowLabel == 'warningLeft':
        return 2
    elif rowLabel == 'stop' or rowLabel == 'stopLeft':
        return 3
    else:
        return None

combined_df['signal'] = combined_df['signal'].apply(changeSignal)
combined_df

In [ ]:
sorted(combined_df['signal'].unique())

In [ ]:
# Need to change the id name to fit clip name in datasets
# dayTraining/dayClip1--00000.jpg ----> C:\Users\Austin\HCL_TrafficLight\LISATrafficSet\dayTrain\dayTrain\dayClip1\frames\dayClip1--00000
def updateName(fileName):
    
    # First we have to parse id to obtain which clip folder and which specific frame
    folder, frameName = fileName.split('/')
    frameDir, clipId = frameName.split('--')
    
    if folder == 'dayTraining':
        newFileName = dataPath + '/dayTrain/dayTrain/{0}/frames/{1}'.format(frameDir, frameName)
    else:
        newFileName = dataPath + '/nightTrain/nightTrain/{0}/frames/{1}'.format(frameDir, frameName)
    
    
    return newFileName

In [ ]:
combined_df['id']= combined_df.id.apply(updateName)
combined_df.head()

In [ ]:
from IPython.display import display
im="C:/Users/Austin/HCL_TrafficLight/LISATrafficSet/dayTrain/dayTrain/dayClip1/frames/dayClip1--00000.jpg"
display(Image.open(im))

In [ ]:
# img = cv2.imread('C:/Users/Austin/HCL_TrafficLight/LISATrafficSet/dayTrain/dayTrain/dayClip1/frames/dayClip1--00000.jpg',1)
# img = cv2.resize(img, (0,0), fx=0.5,fy=0.5)
# img = cv2.rotate(img, cv2.cv2.ROTATE_90_COUNTERCLOCKWISE)

# cv2.imshow('Image', img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
annoTags = combined_df['signal'].unique()
annoTags

In [ ]:
# def draw_bbox(img, boxes):
#     boxes = boxes.split()
#     boxes = [512 * float(i) for i in boxes]
#     y = boxes[0]# * 512
#     x = boxes[1]# * 512
#     h = (boxes[2] - boxes[0])# * 512
#     w = (boxes[3] - boxes[1])# * 512
#     shape = [(x, y), (w, h)]
#     print(boxes)

In [ ]:
def sampleBoxes(df):
    fig, ax = plt.subplots(3, 1, figsize=(15,15))

    for i, tag in enumerate(annoTags):
    #   Take random sample from dataset to annotate
        sample = df[df['signal']==tag].sample(1)
        bbox = sample[['left_x', 'left_y','right_x','right_y']].values[0]

    #   Change color of image to fit RGB color space
        image = cv2.imread(sample.id.values[0])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    #   Change color of box depending on signal value
        if sample['signal'].values[0] == 1:
            boxColor = (0,255,0)
        elif sample['signal'].values[0] == 2:
            boxColor = (255,255,0)
        elif sample['signal'].values[0] == 3:
            boxColor = (255,0,0)

        cv2.rectangle(image, 
                      (bbox[0],bbox[1]), 
                      (bbox[2],bbox[3]), 
                      boxColor, 2)

        ax[i].set_title(idx_to_label[tag])
        ax[i].set_axis_off()
        ax[i].imshow(image)

In [ ]:
sampleBoxes(combined_df)

In [ ]:
def nameSplit(frameId):
    clipName = frameId.split('/')[7]
    return clipName

combined_df['clipNames'] = sorted(combined_df['id'].apply(nameSplit))
combined_df['clipNames'].unique()

In [ ]:
# We need to split our dataframe into test and training sets
def splitTestTrain(df):
    clipNames = combined_df['clipNames'].unique()
    nightClips = []
    dayClips = []

    for name in clipNames:
        if 'day' in name:
            dayClips.append(name)
        else:
            nightClips.append(name)

    # Select random clip names from day and night folders to use for test set
    testNightClipNames = random.choices(nightClips, k=int(len(nightClips)*0.25))
    testDayClipNames = random.choices(dayClips, k=int(len(dayClips)*0.25))
    testClipNames = testDayClipNames + testNightClipNames

    train_df = combined_df[~combined_df['clipNames'].isin(testClipNames)]
    test_df = combined_df[combined_df['clipNames'].isin(testClipNames)]

    return train_df, test_df

In [ ]:
train_df, test_df = splitTestTrain(combined_df)

In [ ]:
train_df.shape[0]/combined_df.shape[0]

In [ ]:
train_df, valid_df = splitTestTrain(train_df)

In [ ]:
print("Train: {0}, Valid: {1}".format(train_df.shape,valid_df.shape))

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

In [ ]:
class custom_traffic_lights:
    def __init__(self, df, transforms=None):
        self.df = df
        self.dim = (224,224)
        self.ids = df.id.unique()
        self.transforms = transforms

    def __getitem__(self, idx):
        
        imageID = self.id[idx]
        
        
        landmarkid = self.df.iloc[idx].id
        url = self.df[self.df.id == id].url.values[0]
        boxes = self.df[self.df.id == id].box.values[0]
        print("enters get item with id " + str(id))

        boxes = self.format_boxes(boxes, self.dim)


        target = {}
        target["boxes"] = torch.as_tensor([boxes], dtype=torch.int64)
        #target["labels"] = torch.as_tensor(labels, dtype=torch.int64)
        target["image_id"] = torch.tensor([idx])

        image = self.url_to_image(url, self.dim)
        return image, target

    def __len__(self):
        return self.id.shape[0]